# Data importing

In [2]:
import numpy as np

x = [[1, 0.6, 0.1],
     [0, -0.4, 0.8],
     [0, 0.2, 0.5],
     [1, 0.4, -0.1]]

In [3]:
coef1 = 0.5
coef2 = 0.5

p1 = 0.3
p2 = 0.7

mean1 = [1, 1]
cov_matrix1 = [[2, 0.5], [0.5, 2]]

mean2 = [0, 0]
cov_matrix2 = [[1.5, 1], [1, 1.5]]

# E-Step

P(x|c)

In [4]:
from scipy.stats import multivariate_normal

p_x_c1 = []
for i in range(len(x)):
    if x[i][0] == 1:
        p = 0.3
    else:
        p = 0.7
    p_x_c1.append(p * multivariate_normal.pdf(x[i][1:], mean1, cov_matrix1))

p_x_c1

[0.019972589760910178,
 0.035034221769896304,
 0.047862166487579405,
 0.01771409803311908]

In [5]:
p_c1_x_prop = np.multiply(0.5, p_x_c1)
p_c1_x_prop

array([0.00998629, 0.01751711, 0.02393108, 0.00885705])

In [6]:
p_x_c2 = []
for i in range(len(x)):
    if x[i][0] == 1:
        p = 0.7
    else:
        p = 0.3
    p_x_c2.append(p * multivariate_normal.pdf(x[i][1:], mean2, cov_matrix2))

p_x_c2

[0.08373285999441,
 0.02045717409764841,
 0.03887431044487811,
 0.08715006283612474]

In [7]:
p_c2_x_prop = np.multiply(0.5, p_x_c2)
p_c2_x_prop

array([0.04186643, 0.01022859, 0.01943716, 0.04357503])

P(c|x)

In [8]:
p_c1_x = np.divide(p_c1_x_prop, np.add(p_c1_x_prop, p_c2_x_prop))
p_c1_x

array([0.19258959, 0.63134512, 0.55181128, 0.16892423])

In [9]:
p_c2_x = np.divide(p_c2_x_prop, np.add(p_c1_x_prop, p_c2_x_prop))
p_c2_x

array([0.80741041, 0.36865488, 0.44818872, 0.83107577])

N1 and N2

In [10]:
N1 = sum(p_c1_x)
N1

1.5446702187154808

In [11]:
N2 = sum(p_c2_x)
N2

2.4553297812845187

π1 and π2

In [12]:
new_coef1 = N1/(N1+N2)
new_coef1

0.38616755467887026

In [13]:
new_coef2 = N2/(N1+N2)
new_coef2

0.6138324453211298

p1 and p2

In [14]:
new_p1 = 0
for i in range(len(x)):
    new_p1 += p_c1_x[i] * x[i][0]

new_p1 = new_p1 / N1
new_p1

0.23403948408541084

In [15]:
new_p2 = 0
for i in range(len(x)):
    new_p2 += p_c2_x[i] * x[i][0]

new_p2 = new_p2 / N2
new_p2

0.6673181710330363

mean1 and mean2

In [16]:
new_mean1 = np.array([[0], [0]])
for i in range(len(x)):
    new_mean1 = np.add(new_mean1, p_c1_x[i] * np.vstack(x[i][1:]))

new_mean1 = np.divide(new_mean1, N1)
new_mean1

array([[0.026509  ],
       [0.50712978]])

In [17]:
new_mean2 = np.array([[0], [0]])
for i in range(len(x)):
    new_mean2 = np.add(new_mean2, p_c2_x[i] * np.vstack(x[i][1:]))

new_mean2 = np.divide(new_mean2, N2)
new_mean2

array([[0.30914476],
       [0.2104205 ]])

cov_matrix1 and cov_matrix2

In [18]:
new_covmatrix1 = np.array([[0, 0], [0, 0]])
for i in range(len(x)):
    new_covmatrix1 = np.add(new_covmatrix1, 
        np.dot(
            p_c1_x[i] * np.subtract(np.vstack(x[i][1:]), new_mean1),
            np.transpose(np.subtract(np.vstack(x[i][1:]), new_mean1))
        )
    )

new_covmatrix1 = np.divide(new_covmatrix1, N1)
new_covmatrix1

array([[ 0.14136501, -0.10540546],
       [-0.10540546,  0.0960526 ]])

In [19]:
new_covmatrix2 = np.array([[0, 0], [0, 0]])
for i in range(len(x)):
    new_covmatrix2 = np.add(new_covmatrix2, 
        np.dot(
            p_c2_x[i] * np.subtract(np.vstack(x[i][1:]), new_mean2),
            np.transpose(np.subtract(np.vstack(x[i][1:]), new_mean2))
        )
    )

new_covmatrix2 = np.divide(new_covmatrix2, N2)
new_covmatrix2

array([[ 0.10829305, -0.08865175],
       [-0.08865175,  0.1041233 ]])

In [21]:
np.linalg.inv(new_covmatrix2)

array([[30.4748368 , 25.94661724],
       [25.94661724, 31.69523838]])

# Ex2 - X_new = [1, 0.3, 0.7]

In [25]:
x_new = [1, 0.3, 0.7]

In [35]:
if x_new[0] == 1:
    p = new_p1
else:
    p = 1-new_p1
p_xnew_c1 = p * multivariate_normal.pdf(x_new[1:], mean1, cov_matrix1)

p_xnew_c1

0.016946616264150456

In [34]:
p_c1_xnew_prop = new_coef1 * p_xnew_c1
p_c1_xnew_prop

0.006544289228869047

In [42]:
if x_new[0] == 1:
    p = new_p2
else:
    p = 1-new_p2
p_xnew_c2 = p * multivariate_normal.pdf(x_new[1:], mean2, cov_matrix2)

p_xnew_c2

0.07934600571519558

In [40]:
p_c2_xnew_prop = new_coef2 * p_xnew_c2
p_c2_xnew_prop

0.04870515271462284

In [43]:
p_c1_xnew = p_c1_xnew_prop / np.add(p_c1_xnew_prop, p_c2_xnew_prop)
p_c1_xnew

0.11844987023692338

In [44]:
p_c2_xnew = p_c2_xnew_prop / np.add(p_c1_xnew_prop, p_c2_xnew_prop)
p_c2_xnew

0.8815501297630766